- khai niem yolov8: 
+ la phien ban moi nhat cua xu ly thi giac may tinh va hinh anh
+ duoc xay dung trong deep learning va thi giac may tinh uu tien ve toc do va do chinh xac


* o lab2 nay nhom thuc hien de tai: phat hien va danh gia hieu qua cua cac doi tuong dua vao yolov8 va kerasCv

1. Septup: goi cac thu vien can thiet

In [7]:
import os
from tqdm.auto import tqdm
import xml.etree.ElementTree as ET

import tensorflow as tf
from tensorflow import keras

import keras_cv
from keras_cv import bounding_box
from keras_cv import visualization


2. Load data

In [8]:
SPLIT_RATIO = 0.2
BATCH_SIZE = 4
LEARNING_RATE = 0.001
EPOCH = 5
GLOBAL_CLIPNORM = 10.0


3. Dictionary tao tung ten lop voi ma dinh danh duy nhat

In [10]:
class_ids = [
    "car",
    "pedestrian",
    "trafficLight",
    "biker",
    "truck",
]
class_mapping = dict(zip(range(len(class_ids)), class_ids))

# Path to images and annotations
path_images = "/kaggle/input/dataset/data/images/"
path_annot = "/kaggle/input/dataset/data/annotations/"

# Get all XML file paths in path_annot and sort them
xml_files = sorted(
    [
        os.path.join(path_annot, file_name)
        for file_name in os.listdir(path_annot)
        if file_name.endswith(".xml")
    ]
)

# Get all JPEG image file paths in path_images and sort them
jpg_files = sorted(
    [
        os.path.join(path_images, file_name)
        for file_name in os.listdir(path_images)
        if file_name.endswith(".jpg")
    ]
)


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/dataset/data/annotations/'

3. ham cho phep doc file xml va tim kiem ten anh va path
ham se tra ve 3 gia tri: anh, duong dan, cac khoi block dc bieu thi dang (xmin, ymin, xmax, ymax)


In [ ]:
def parse_annotation(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    image_name = root.find("filename").text
    image_path = os.path.join(path_images, image_name)

    boxes = []
    classes = []
    for obj in root.iter("object"):
        cls = obj.find("name").text
        classes.append(cls)

        bbox = obj.find("bndbox")
        xmin = float(bbox.find("xmin").text)
        ymin = float(bbox.find("ymin").text)
        xmax = float(bbox.find("xmax").text)
        ymax = float(bbox.find("ymax").text)
        boxes.append([xmin, ymin, xmax, ymax])

    class_ids = [
        list(class_mapping.keys())[list(class_mapping.values()).index(cls)]
        for cls in classes
    ]
    return image_path, boxes, class_ids


image_paths = []
bbox = []
classes = []
for xml_file in tqdm(xml_files):
    image_path, boxes, class_ids = parse_annotation(xml_file)
    image_paths.append(image_path)
    bbox.append(boxes)
    classes.append(class_ids)


In [ ]:
classes = [
    [8, 8, 8, 8, 8],      # 5 classes
    [12, 14, 14, 14],     # 4 classes
    [1],                  # 1 class
    [7, 7],               # 2 classes
 ...]


In [ ]:
bbox = [
    [[199.0, 19.0, 390.0, 401.0],
    [217.0, 15.0, 270.0, 157.0],
    [393.0, 18.0, 432.0, 162.0],
    [1.0, 15.0, 226.0, 276.0],
    [19.0, 95.0, 458.0, 443.0]],     #image 1 has 4 objects
    [[52.0, 117.0, 109.0, 177.0]],   #image 2 has 1 object
    [[88.0, 87.0, 235.0, 322.0],
    [113.0, 117.0, 218.0, 471.0]],   #image 3 has 2 objects
 ...]


4. dung tf.rangged.constrant de tao 2 bien bbox va classes chua 2 chuoi o tren
- rangged: co the xu ly do dai du lieu khac nhau theo 1 chieu hay nhieu chieu--> no rat huu ich khi xu ly du lieu co chuoi co do dai thay doi vd nhu van ban, tgian


5. o truong hopj nay 2 ben bbox va classes co do dai khac biet cho moi anh
- dua vao so luong doi tuong trong anh co the dong khung cac doi tuong tuong ung
- sau do tensors rangged nay duoc su dung de tao tf.data.Dataset bang method from_tensor_slices.
--> phuong thuc nay tensors co the xu ly cac do dai du lieu khac nhau voi moi hinh anh duoc cung cap

In [ ]:
bbox = tf.ragged.constant(bbox)
classes = tf.ragged.constant(classes)
image_paths = tf.ragged.constant(image_paths)

data = tf.data.Dataset.from_tensor_slices((image_paths, classes, bbox))


6. tach du lieu

In [ ]:
# Determine the number of validation samples
num_val = int(len(xml_files) * SPLIT_RATIO)

# Split the dataset into train and validation sets
val_data = data.take(num_val)
train_data = data.skip(num_val)


7. de gioi han dinh dang dong hop cac hop gioi han trong Keras phai do dinh dang xac dinh truoc vi vay phai dong goi hop vao dictionnary nhu code ben duoi

8. dictionary co 2 keys: 'boxes' va 'classes' moi keys anh xa toi TensorFlow RaggedTensor or Tensor object. 
trong do:
+ boxes: (bacth va num_boxes) la noi chua so luong toi da gioi han hop trong bat ki anh nao, 4 dai dien cho 4 gia tri can thiet de xac dinh xmin,ymin,xmax,ymax
+ classes: (batch va num_boxes) dai dien cho label cho hop gioi han trong boxes, kich thuoc cua hop khong deu nhau nghiax la so luong hop co the khac nhau

In [ ]:
bounding_boxes = {
    # num_boxes may be a Ragged dimension
    'boxes': Tensor(shape=[batch, num_boxes, 4]),
    'classes': Tensor(shape=[batch, num_boxes])
}


In [ ]:
{"images": images, "bounding_boxes": bounding_boxes}


In [ ]:
def load_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    return image


def load_dataset(image_path, classes, bbox):
    # Read Image
    image = load_image(image_path)
    bounding_boxes = {
        "classes": tf.cast(classes, dtype=tf.float32),
        "boxes": bbox,
    }
    return {"images": tf.cast(image, tf.float32), "bounding_boxes": bounding_boxes}


9. o day tao cac layer de resizes img thanh 640x640 pixels trong khi van duy tri ti le goc, cac hop goi han lien ket voi hinh anh duoc chi dinh dang xyxy. 

In [ ]:
boxes = keras_cv.bounding_box.convert_format(
        bounding_box,
        images=image,
        source="xyxy",  # Original Format
        target="xywh",  # Target Format (to which we want to convert)
    )


* Data Augmentation

10. khi xay dung pipline dectection dieu kho khan nhat la viec tang cuong du lieu. no lien quan den viec ap dung phep bien doi khac nhau de tang tinh da dang cua du lieu va kha nang khai quat cua mo hinh
- Keras cung cap cac ho tro cho viec tang cuong hop gioi han. KerasCV cung cap mot bo suu tap phong phu cac lop tang cuongdu lieu de xu ly cac hop gioi han
- cac lop nay dieu chinh toa do hop mot cach thong minh khi hinh anh duoc chuyen doi de dam bao can chinh chinh xac hinh anh\
- tan dung kha nang cua KerasCV co the thuc hien tang cuong nhanh chong trong pipeline tf.data, no se tro nen lien mach va hieu qua cho phe train tot hon

In [ ]:
augmenter = keras.Sequential(
    layers=[
        keras_cv.layers.RandomFlip(mode="horizontal", bounding_box_format="xyxy"),
        keras_cv.layers.RandomShear(
            x_factor=0.2, y_factor=0.2, bounding_box_format="xyxy"
        ),
        keras_cv.layers.JitteredResize(
            target_size=(640, 640), scale_factor=(0.75, 1.3), bounding_box_format="xyxy"
        ),
    ]
)


* tao tranning dataset

In [ ]:
train_ds = train_data.map(load_dataset, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.shuffle(BATCH_SIZE * 4)
train_ds = train_ds.ragged_batch(BATCH_SIZE, drop_remainder=True)
train_ds = train_ds.map(augmenter, num_parallel_calls=tf.data.AUTOTUNE)


* Creating Validation Dataset

In [ ]:
resizing = keras_cv.layers.JitteredResize(
    target_size=(640, 640),
    scale_factor=(0.75, 1.3),
    bounding_box_format="xyxy",
)

val_ds = val_data.map(load_dataset, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.shuffle(BATCH_SIZE * 4)
val_ds = val_ds.ragged_batch(BATCH_SIZE, drop_remainder=True)
val_ds = val_ds.map(resizing, num_parallel_calls=tf.data.AUTOTUNE)


* Visualization

In [ ]:
def visualize_dataset(inputs, value_range, rows, cols, bounding_box_format):
    inputs = next(iter(inputs.take(1)))
    images, bounding_boxes = inputs["images"], inputs["bounding_boxes"]
    visualization.plot_bounding_box_gallery(
        images,
        value_range=value_range,
        rows=rows,
        cols=cols,
        y_true=bounding_boxes,
        scale=5,
        font_scale=0.7,
        bounding_box_format=bounding_box_format,
        class_mapping=class_mapping,
    )


visualize_dataset(
    train_ds, bounding_box_format="xyxy", value_range=(0, 255), rows=2, cols=2
)

visualize_dataset(
    val_ds, bounding_box_format="xyxy", value_range=(0, 255), rows=2, cols=2
)


* ta can xac dinh dau vao tu preprocesssing dictionary va lay no de san san dua vao model

In [ ]:
def dict_to_tuple(inputs):
    return inputs["images"], inputs["bounding_boxes"]


train_ds = train_ds.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)

val_ds = val_ds.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.prefetch(tf.data.AUTOTUNE)


* Creating Model
- YOLOv8 là mô hình YOLO tiên tiến được sử dụng cho nhiều tác vụ thị giác máy tính khác nhau, chẳng hạn như phát hiện đối tượng, phân loại hình ảnh và phân đoạn phiên bản. 
- Ultralytics, người sáng tạo ra YOLOv5, cũng đã phát triển YOLOv8, kết hợp nhiều cải tiến và thay đổi về kiến ​​trúc cũng như trải nghiệm của nhà phát triển so với phiên bản tiền nhiệm. 
- YOLOv8 là model tiên tiến mới nhất được đánh giá cao trong ngành.

1. With Pre-trained coco weight:

In [ ]:
backbone = keras_cv.models.YOLOV8Backbone.from_preset(
    "yolo_v8_s_backbone_coco"  # We will use yolov8 small backbone with coco weights
)


- tiep theo xay dung yoloV8 model bang cach su dung YOLOV8Detector 
* trong do
+ num_classes: chi dinh so luong lop doi tuong can phat hien dua tren kich thuoc danh sach class_mapping
+ bounding_box_form: thong bao cho mo hinh ve dinh dang cuar bbox trong tap du lieu
+fpn_depth: do sau 

In [ ]:
yolo = keras_cv.models.YOLOV8Detector(
    num_classes=len(class_mapping),
    bounding_box_format="xyxy",
    backbone=backbone,
    fpn_depth=1,
)


* Compile the Model
- Classification Loss: ham nay tinh toan su khac biet giua xac suat lop du doan va xac suat lop trong thuc te, trong truong hop nay 'binary_crossebtropy' la giai phap phan loai nhi phan duoc su dung,
- Box Loss: dung do luong su khac biet giua hop gioi han duoc du doan, truong hop nay so lieu Complete IoU(CloU) khong chi do su chong cheo giua cac hop gioi han dc du doan thuc te maf con xem xet su khac biet ve ti le khung hinh

In [ ]:
optimizer = tf.keras.optimizers.Adam(
    learning_rate=LEARNING_RATE,
    global_clipnorm=GLOBAL_CLIPNORM,
)

yolo.compile(
    optimizer=optimizer, classification_loss="binary_crossentropy", box_loss="ciou"
)


* COCO Metric Callback
dung BoxCOCOMetrics tu KerasCV de danh gia mo hinh va tinh do chinh xac trung binh

In [ ]:
class EvaluateCOCOMetricsCallback(keras.callbacks.Callback):
    def __init__(self, data, save_path):
        super().__init__()
        self.data = data
        self.metrics = keras_cv.metrics.BoxCOCOMetrics(
            bounding_box_format="xyxy",
            evaluate_freq=1e9,
        )

        self.save_path = save_path
        self.best_map = -1.0

    def on_epoch_end(self, epoch, logs):
        self.metrics.reset_state()
        for batch in self.data:
            images, y_true = batch[0], batch[1]
            y_pred = self.model.predict(images, verbose=0)
            self.metrics.update_state(y_true, y_pred)

        metrics = self.metrics.result(force=True)
        logs.update(metrics)

        current_map = metrics["MaP"]
        if current_map > self.best_map:
            self.best_map = current_map
            self.model.save(self.save_path)  # Save the model when mAP improves

        return logs


* train model

In [ ]:
yolo.fit(
    train_ds,
    validation_data=val_ds,
    epochs=3,
    callbacks=[EvaluateCOCOMetricsCallback(val_ds, "model.h5")],
)


* visual Predictions

In [ ]:
def visualize_detections(model, dataset, bounding_box_format):
    images, y_true = next(iter(dataset.take(1)))
    y_pred = model.predict(images)
    y_pred = bounding_box.to_ragged(y_pred)
    visualization.plot_bounding_box_gallery(
        images,
        value_range=(0, 255),
        bounding_box_format=bounding_box_format,
        y_true=y_true,
        y_pred=y_pred,
        scale=4,
        rows=2,
        cols=2,
        show=True,
        font_scale=0.7,
        class_mapping=class_mapping,
    )


visualize_detections(yolo, dataset=val_ds, bounding_box_format="xyxy")
